In [1]:
print("hello workd")

hello workd


In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
os.environ["java_home"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["spark_home"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,553 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,171 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,133 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,867 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:14 http://arc

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col,explode
spark = SparkSession.builder.appName("Collaborative filtering").getOrCreate()

In [17]:
moviesDF = spark.read.options(header="True", inferSchema="True").csv("/content/movies.csv")
ratingsDF = spark.read.options(header="True", inferSchema="True").csv("/content/ratings.csv")

In [18]:
display(moviesDF)
display(ratingsDF)

DataFrame[movieId: int, title: string, genres: string]

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [19]:
ratings = ratingsDF.join(moviesDF, 'movieId', 'left')

In [20]:
(train, test) = ratings.randomSplit([0.8,0.2])

In [21]:
ratings.count()

100836

In [22]:
print(train.count())
train.show()

80557
+-------+------+------+----------+----------------+--------------------+
|movieId|userId|rating| timestamp|           title|              genres|
+-------+------+------+----------+----------------+--------------------+
|      1|     5|   4.0| 847434962|Toy Story (1995)|Adventure|Animati...|
|      1|     7|   4.5|1106635946|Toy Story (1995)|Adventure|Animati...|
|      1|    15|   2.5|1510577970|Toy Story (1995)|Adventure|Animati...|
|      1|    18|   3.5|1455209816|Toy Story (1995)|Adventure|Animati...|
|      1|    31|   5.0| 850466616|Toy Story (1995)|Adventure|Animati...|
|      1|    32|   3.0| 856736119|Toy Story (1995)|Adventure|Animati...|
|      1|    33|   3.0| 939647444|Toy Story (1995)|Adventure|Animati...|
|      1|    40|   5.0| 832058959|Toy Story (1995)|Adventure|Animati...|
|      1|    43|   5.0| 848993983|Toy Story (1995)|Adventure|Animati...|
|      1|    44|   3.0| 869251860|Toy Story (1995)|Adventure|Animati...|
|      1|    45|   4.0| 951170182|Toy Story (

In [23]:
print(test.count())
test.show()

20279
+-------+------+------+----------+----------------+--------------------+
|movieId|userId|rating| timestamp|           title|              genres|
+-------+------+------+----------+----------------+--------------------+
|      1|     1|   4.0| 964982703|Toy Story (1995)|Adventure|Animati...|
|      1|    17|   4.5|1305696483|Toy Story (1995)|Adventure|Animati...|
|      1|    19|   4.0| 965705637|Toy Story (1995)|Adventure|Animati...|
|      1|    21|   3.5|1407618878|Toy Story (1995)|Adventure|Animati...|
|      1|    27|   3.0| 962685262|Toy Story (1995)|Adventure|Animati...|
|      1|    57|   5.0| 965796031|Toy Story (1995)|Adventure|Animati...|
|      1|    68|   2.5|1158531426|Toy Story (1995)|Adventure|Animati...|
|      1|    93|   3.0| 942767337|Toy Story (1995)|Adventure|Animati...|
|      1|   119|   3.5|1435942468|Toy Story (1995)|Adventure|Animati...|
|      1|   121|   4.0| 847656180|Toy Story (1995)|Adventure|Animati...|
|      1|   130|   3.0| 832589610|Toy Story (

als model -  is a popular algorithm used for collaborative filtering in recommendation systems


In [24]:
als = ALS(userCol = "userId", itemCol="movieId", ratingCol="rating", nonnegative=True,implicitPrefs=False, coldStartStrategy="drop")
#usercol - refers to the user
#itemcol - refers to the item selected or the item of which recommendation should be made
#ratingcol - the parameter according to which ratings are defined
#nonnegative - consideration of negative ratings
#implicitperfs - refers to the implicit or explicit dataset
#colstartstrategy - dropping the users with no ratings

hyperparameter tuning and cross validation

In [25]:
#Hyperparameter tuning is the process of finding the best settings for the parts of a machine learning model that you have to decide before training
#When tuning hyperparameters, you find the best combination of hyperparameters that result in the highest performance of your model.
#To do this reliably, you use cross-validation to evaluate the performance of each combination of hyperparameter

In [26]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

#is a utility in PySpark's MLlib that helps you create a grid of hyperparameters for model tuning, particularly for use with
#this will create 4*4 = 16 models with all combinations of parameters with values and then refer to the best module using cross validator
#rank parameter, number of features used in the model, model will be evaluated based on the values in the list
#regulizer parameter, which prevents the model from overfitting

In [27]:
evaluator = RegressionEvaluator(
           metricName="rmse",     #this shows the parameters with which evaluation should be performed
           labelCol="rating",     #rmse, refers to root mean square error
           predictionCol="prediction")
#regression evaluator along with param grid builder goes for cross validator

In [28]:
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)
#to get the optimised or best cross validator

In [29]:
model = cv.fit(train)
best_model = model.bestModel #getting the best models created inside param grid
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.8760148688962565


In [30]:
print(RMSE)

0.8760148688962565


In [31]:
recommendations = best_model.recommendForAllUsers(5)
#it will recommend top 5 best models for users based on ratings for users

In [32]:
df = recommendations
display(df)

DataFrame[userId: int, recommendations: array<struct<movieId:int,rating:float>>]

In [33]:
df2 = df.withColumn("movieid_rating", explode("recommendations"))
display(df2)

DataFrame[userId: int, recommendations: array<struct<movieId:int,rating:float>>, movieid_rating: struct<movieId:int,rating:float>]

In [34]:
display(df2.select("userId", col("movieid_rating.movieId"), col("movieid_rating.rating")))

DataFrame[userId: int, movieId: int, rating: float]